Peter Schnizer


# HW 3
---

 ### Task 1:
Create your own PyTorch class that implements the method of SCAD regularization and variable selection (smoothly clipped absolute deviations) for linear models. Your development should be based on the following references:

https://andrewcharlesjones.github.io/journal/scad.html

https://www.jstor.org/stable/27640214?seq=1

Test your method one a real data set, and determine a variable selection based on features importance according to SCAD.


**Answer:**

In [181]:
import torch # we are going to use pytorch instead of numpy because it's much faster.
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.datasets import make_spd_matrix
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as R2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


device = torch.device("cpu")
dtype = torch.float32

In [182]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss(reduction='mean')(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1)
        l2_reg = torch.norm(self.linear.weight, p=2)

        objective = (1/2) * mse_loss + self.alpha * (
            self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2)

        return objective

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X) # a forward propagation
            loss = self.loss(y_pred, y)
            loss.backward() # the optimizer computes the gradient
            optimizer.step() # the optimizer updates the weights according to the chosen heuristics

            if (epoch + 1) % 10 == 0: # decide to display the progress
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight


In [183]:
class SCAD(nn.Module):
    def __init__(self, input_size, alpha=0.5, lmda=0.5):
        super(SCAD, self).__init__()
        self.input_size = input_size
        self.alpha = alpha
        self.lmda = lmda

        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the SCAD model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)
    
    def scad_penalty(self):
        is_linear = (torch.abs(self.linear.weight) <= self.lmda)
        is_quadratic = torch.logical_and(self.lmda < torch.abs(self.linear.weight), torch.abs(self.linear.weight) <= self.alpha * self.lmda)
        is_constant = (self.alpha * self.lmda) < torch.abs(self.linear.weight)

        linear_part = self.lmda * torch.abs(self.linear.weight) * is_linear
        quadratic_part = (2 * self.alpha * self.lmda * torch.abs(self.linear.weight) - self.linear.weight**2 - self.lmda**2) / (2 * (self.alpha - 1)) * is_quadratic
        constant_part = (self.lmda**2 * (self.alpha + 1)) / 2 * is_constant
        return linear_part + quadratic_part + constant_part

    def loss(self, y_pred, y_true):
        """
        Compute the SCAD loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The SCAD loss.

        """
        mse_loss = nn.MSELoss(reduction='mean')(y_pred, y_true)
        scad_reg = torch.norm(self.scad_penalty(), p=1)
        objective = mse_loss + scad_reg
        return objective

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the SCAD model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X) # a forward propagation
            loss = self.loss(y_pred, y)
            loss.backward() # the optimizer computes the gradient
            optimizer.step() # the optimizer updates the weights according to the chosen heuristics

            if (epoch + 1) % 10 == 0: # decide to display the progress
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight


In [184]:
# Test on nba dataset
nba = pd.read_csv('Data/nba.csv')
X = nba.drop(['Date','Matchup','Spread','Margin'],axis=1).to_numpy()[:-2]
y = nba['Margin'].to_numpy()[:-2]
X_cols = nba.drop(['Date','Matchup','Spread','Margin'],axis=1).columns
X_cols

Index(['PPG', 'Rebs', 'Stls', 'Blks', 'TOVs', 'FG%', '3FG%', 'Off Rtg',
       'Def Rtg', 'Pace', 'Lost Prod'],
      dtype='object')

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

scad_model = SCAD(input_size=X.shape[1], alpha=0.1, lmda=0.2)
scad_model.fit(X_train_tensor, y_train_tensor)

enet_model = ElasticNet(input_size=X.shape[1], alpha=0.1, l1_ratio=0.1)

scad_pred = scad_model.predict(X_test_tensor)
enet_pred = enet_model.predict(X_test_tensor)

Epoch [10/100], Loss: 195.93380737304688
Epoch [20/100], Loss: 194.15472412109375
Epoch [30/100], Loss: 193.335205078125
Epoch [40/100], Loss: 192.94654846191406
Epoch [50/100], Loss: 192.76776123046875
Epoch [60/100], Loss: 192.69390869140625
Epoch [70/100], Loss: 192.6868438720703
Epoch [80/100], Loss: 192.68145751953125
Epoch [90/100], Loss: 192.6739044189453
Epoch [100/100], Loss: 192.66575622558594


c:\Users\ptsch\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([627])) that is different to the input size (torch.Size([627, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [219]:
print(f'SCAD MSE: {mse(y_test, scad_pred)}\nElasticNet MSE: {mse(y_test, enet_pred)}')

SCAD MSE: 198.3649080353595
ElasticNet MSE: 202.7778594511267


The SCAD regularization term works better than ElasticNet for the NBA data!

In [220]:
coeffs = scad_model.get_coefficients()[0]
ranked_coeffs_idxs = torch.abs(coeffs).argsort(descending=True)
ranked_ftrs = np.array(X_cols)[ranked_coeffs_idxs]
print('Feature Rankings:')
for i, (idx, feature) in enumerate(zip(ranked_coeffs_idxs, ranked_ftrs)):
    print(str(i+1)+':', feature,  coeffs.tolist()[idx])

Feature Rankings:
1: Off Rtg 0.5143522620201111
2: Stls 0.513846755027771
3: PPG 0.48982667922973633
4: FG% 0.472359299659729
5: 3FG% 0.43158531188964844
6: Pace 0.3675922751426697
7: Blks 0.2889552116394043
8: Def Rtg 0.2420038878917694
9: Rebs 0.15583162009716034
10: Lost Prod 0.13700784742832184
11: TOVs 0.13058532774448395


The next feature selection I would try based off of these results would be minimizing the model to just Off Rtg, Stls, PPG, FG%, and 3FG%.

---
### Task 2:
Based on the simulation design explained in class, generate 500 data sets where the input features have a strong correlation structure (you may consider a 0.9) and apply ElasticNet, SqrtLasso and SCAD to check which method produces the best approximation of an ideal solution, such as a "betastar" you design with a sparsity pattern.

**Answer:**

In [221]:
class SqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        """
        Initialize the  regression model.


        """
        super(SqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha


        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1,bias=False,device=device,dtype=dtype)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss(reduction='mean')(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = torch.sqrt(mse_loss) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=500, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

In [235]:
def make_correlated_features(num_samples,p,rho):
  vcor = [] 
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x

p = 200
n = 500
rho = 0.9

In [236]:
x = make_correlated_features(n,p,rho)
x.shape

(500, 200)

In [237]:
beta =np.array([0, -1,2,3,0,0,0,0,2,-1,4])
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)

In [238]:
y = x@betastar + 1.5*np.random.normal(size=(n,1))

In [239]:
x_torch = torch.tensor(x,device=device, dtype=torch.float32)
y_torch = torch.tensor(y,device=device, dtype=torch.float32)

In [240]:
enet_model = ElasticNet(input_size=x_torch.shape[1],alpha=0.1,l1_ratio=0.5)
scad_model = SCAD(input_size=x_torch.shape[1],alpha=0.1,lmda=0.5)
sqrtlas_model = SqrtLasso(input_size=x_torch.shape[1],alpha=0.1)

enet_model.fit(x_torch,y_torch, num_epochs=10000, learning_rate=0.01)
scad_model.fit(x_torch,y_torch, num_epochs=10000, learning_rate=0.01)
sqrtlas_model.fit(x_torch,y_torch, num_epochs=10000, learning_rate=0.01)

Epoch [10/10000], Loss: 20.341869354248047
Epoch [20/10000], Loss: 14.503839492797852
Epoch [30/10000], Loss: 10.744985580444336
Epoch [40/10000], Loss: 8.270544052124023
Epoch [50/10000], Loss: 6.648612976074219
Epoch [60/10000], Loss: 5.555250644683838
Epoch [70/10000], Loss: 4.793609619140625
Epoch [80/10000], Loss: 4.264492988586426
Epoch [90/10000], Loss: 3.8985838890075684
Epoch [100/10000], Loss: 3.6425962448120117
Epoch [110/10000], Loss: 3.4643428325653076
Epoch [120/10000], Loss: 3.3425872325897217
Epoch [130/10000], Loss: 3.253056764602661
Epoch [140/10000], Loss: 3.1857829093933105
Epoch [150/10000], Loss: 3.133524179458618
Epoch [160/10000], Loss: 3.0889980792999268
Epoch [170/10000], Loss: 3.0499088764190674
Epoch [180/10000], Loss: 3.0148208141326904
Epoch [190/10000], Loss: 2.9829249382019043
Epoch [200/10000], Loss: 2.951894760131836
Epoch [210/10000], Loss: 2.922621488571167
Epoch [220/10000], Loss: 2.89424729347229
Epoch [230/10000], Loss: 2.8668668270111084
Epoch [2

In [241]:
enet_pred = enet_model.predict(x_torch)
scad_pred = scad_model.predict(x_torch)
sqrtlas_pred = sqrtlas_model.predict(x_torch)

In [242]:
mse(enet_pred, y), mse(scad_pred, y), mse(sqrtlas_pred, y)

(2.329589279941096, 2.30380426512322, 2.3230125629204164)

It appears that SCAD regularization produces the best approximation.

---
### Task 3
Host your project on your GitHub page.

**Answer:**
https://github.com/Pschnizer/DATA441/blob/main/DATA_441_Project_3.ipynb